<a href="https://colab.research.google.com/github/EstebanAG1005/Proyecto-Data-Science/blob/main/An%C3%A1lisis_Exploratorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Carga y Revisión Inicial

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic
import cv2

from google.colab import drive
import pandas as pd
import zipfile

# Mount the Google Drive
drive.mount('/content/drive')

# Path to the zip file
path_imgs = "/content/drive/MyDrive/Proyecto-Data-Science/dfl-bundesliga-data-shootout.zip"

# Extract the zip file
with zipfile.ZipFile(path_imgs, 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Assuming the train.csv is directly inside the zip, read the CSV file
df = pd.read_csv('/content/train.csv')

# Visualizar las primeras y últimas filas
print('Primeras Filas')
print(df.head())
print('\n\n\nUltimas Filas')
print(df.tail())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


OSError: ignored

# 2.Valores Faltantes

In [ ]:
df.event.value_counts()

In [ ]:
# Valores faltantes
print(df.isnull().sum())

In [ ]:
fig_1 = px.bar(df,x='event',y='video_id',
             color='event_attributes',title = 'Event and Event attributes types')
fig_1.show()

In [ ]:
Crosstable = pd.crosstab(df['event'], df['event_attributes'])
Crosstable.style.background_gradient(cmap='Greens')

In [ ]:
fig_0 = px.bar(df['video_id'].value_counts(),text_auto='.2s', orientation='h',\
               title='Number of events for each video')
fig_0.show()


In [ ]:
Crosstable = pd.crosstab(df['event'], df['event_attributes'])
Crosstable.style.background_gradient(cmap='Greens')

In [ ]:
fig_2 = px.bar(df,x='video_id',y='event',
             color='event',title = 'Event types throw videos')
fig_2.show()

In [ ]:
fig_3 = px.bar(df.query('event not in ["start","end"]')\
    .groupby('video_id')['event'].value_counts().unstack(),orientation='h')
fig_3.show()

# 3. Distribución de los Eventos y Atributos

In [ ]:
# Distribución de eventos
sns.countplot(data=df, x='event')
plt.title('Distribución de Eventos')
plt.show()

# Distribución de atributos (10 más comunes, debido a la variedad)
top_attributes = df['event_attributes'].value_counts().head(10)
top_attributes.plot(kind='bar', color='lightgreen')
plt.title('Top 10 Atributos de Eventos')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


# 4. Distribución del Tiempo

In [ ]:
# Histograma del tiempo
sns.histplot(df['time'], kde=True, bins=50)
plt.title('Distribución del Tiempo de Eventos')
plt.show()


# 5. Relaciones entre las Variables

In [ ]:
# Boxplot del tiempo agrupado por eventos
sns.boxplot(data=df, x='event', y='time')
plt.title('Distribución del Tiempo por Evento')
plt.show()


# 6. Patrones en Atributos según Eventos#

In [ ]:
# Limpiar la columna 'event_attributes'
df['event_attributes'] = df['event_attributes'].str.replace("[", "").str.replace("]", "").str.strip()

# Convertir 'event_attributes' en una lista y expandirla
df_expanded = df.assign(event_attributes=df['event_attributes'].str.split(',')).explode('event_attributes')

# Limpiar espacios adicionales que pueden haber quedado alrededor de los atributos después de dividir
df_expanded['event_attributes'] = df_expanded['event_attributes'].str.strip("'").str.strip()

# Visualizar la frecuencia de atributos por tipo de evento
plt.figure(figsize=(15, 7))
sns.countplot(data=df_expanded, x='event_attributes', hue='event', order=df_expanded['event_attributes'].value_counts().index)
plt.title('Frecuencia de Atributos por Tipo de Evento')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


# 7. Análisis de Secuencias

In [ ]:
# Generar secuencias de eventos y contarlas
df['next_event'] = df['event'].shift(-1)
sequences = df[df['event'] == 'start']['next_event'].value_counts()
print(sequences)

# Visualización
sequences.plot(kind='bar', color='lightblue')
plt.title('Eventos más Comunes después de "start"')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
df['time_end'] = df.time.shift(-1)

In [ ]:
df['time_start'] = df.time.shift(1)

In [ ]:
df = df.dropna(subset=['event_attributes'])

In [ ]:
df = df.reset_index().drop(columns=['index'])

In [ ]:
# Visualizar las primeras y últimas filas
print('Primeras Filas')
print(df.head())
print('\n\n\nUltimas Filas')
print(df.tail())

In [ ]:
df.shape

In [ ]:
sub = pd.read_csv('../input/dfl-bundesliga-data-shootout/sample_submission.csv')
sub.head()

In [ ]:
fig_4 = px.bar(sub,x='event',y='video_id',color = 'event',title = 'Event types and the score')
fig_4.show()

In [ ]:
# look at a data for one video
sub.query('video_id =="019d5b34_0"& event not in ["start","end"]')

In [ ]:
#Code by Baris Dincer https://www.kaggle.com/brsdincer/human-evolution-process-dcgan/notebook
def video_list(Video_Path):
    Video_IMG_List = []

    Video_Cap = cv2.VideoCapture(Video_Path)


    while Video_Cap.isOpened():


        ret,frame = Video_Cap.read()

        if ret != True:
            break

        if Video_Cap.isOpened():
            Transformation_IMG = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            Resize_IMG = cv2.resize(Transformation_IMG,(180,180))
            Video_IMG_List.append(Resize_IMG)


    Video_Cap.release()
    return Video_IMG_List

In [ ]:
video_file = '../input/dfl-bundesliga-data-shootout/test/019d5b34_0.mp4'
video_one_list = video_list(video_file)

In [ ]:
print(np.shape(np.asarray(video_one_list)))

In [ ]:
figure = plt.figure(figsize=(8,8))

Pick_IMG = video_one_list[100]
plt.xlabel(Pick_IMG.shape)
plt.ylabel(Pick_IMG.size)
plt.imshow(Pick_IMG);

In [ ]:
fig,axis = plt.subplots(4,4,figsize=(10,10))

for i,ax in enumerate(axis.flat):

    IMG_From_List = video_one_list[i]
    ax.set_xlabel(IMG_From_List.shape)
    ax.imshow(IMG_From_List)

plt.tight_layout()
plt.show()

In [ ]:
# reference : https://www.kaggle.com/coldfir3/eda-helmet-keypoint-tracking-data-comparison
import PIL.Image
def get_frame_from_video(video_path, frame):
    video_path = video_path
    frame = frame - 1

    !ffmpeg \
        -hide_banner \
        -loglevel fatal \
        -nostats \
        -i $video_path -vf "select=eq(n\,$frame)" -vframes 1 frame.jpg

    img = PIL.Image.open('frame.jpg')#StackOverflow https://stackoverflow.com/questions/10748822/img-image-openfp-attributeerror-class-image-has-no-attribute-open
    os.remove('frame.jpg')
    return img

In [ ]:
get_frame_from_video('../input/dfl-bundesliga-data-shootout/test/019d5b34_0.mp4', 110)

In [ ]:
#Code by Ducky https://www.kaggle.com/code/illgamhoduck/nfl-starter-eda

from IPython.display import Video, display

def video(video_path, ratio=0.7):
    nfl_video = Video(video_path,
                      embed=True,
                      height=int(720 * ratio),
                      width=int(1280 * ratio))
    return nfl_video

video('../input/dfl-bundesliga-data-shootout/test/019d5b34_0.mp4')

# 8. Split the los juegos completos en videos pequeños que despues de eso serán convertidos en frames

In [ ]:
!pip install moviepy

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import shutil
import os

for index, row in df.iterrows():
    video_name = row['event'] + "_" + row['video_id'] + "_" + str(row['time_start']) + "_" + str(row['time_end']) + ".mp4"
    ffmpeg_extract_subclip("train/" + row['video_id'] + ".mp4", row['time_start'], row['time_end'], targetname= video_name)
    shutil.move(os.path.abspath(video_name), "train/" + row['event'])